In [34]:
import pandas as pd
import numpy as np

# data processing

In [35]:
# get csv
# df_long = pd.read_csv("data/exp_TMS.csv")


# files = [
#     "data/LIFESPAN_SZcontrols.csv", 
#     "data/LIFESPAN_agingAZ_all.csv", 
#     "data/LIFESPAN_sommerville.csv", 
#     "data/LIFESPAN_students.csv",
#     "data/TMS_horizonTask.csv",
# ]

# files = [
#     "data/exp_TMS.csv", 
#     "data/exp_all.csv", 
# ]
# df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

df_all = pd.read_csv("data/exp_all_filenames.csv")
df_TMS = pd.read_csv("data/exp_TMS.csv")
df_TMS["file_name"] = "exp_TMS.csv"

df_TMS['subject'] = df_TMS['subject'].astype(str) + '_TMS'
df_all['subject'] = df_all['subject'].astype(str)
df = pd.concat([df_all, df_TMS], ignore_index=True)

In [36]:
df["trial"] = df["trial"].astype(int)

# Rename value_option columns to match original format (m1, m2)
df = df.rename(columns={"value_option0": "m1", "value_option1": "m2"})

# Redefine pivoting function
def pivot_feature(df, feature, prefix):
    return df.pivot_table(
        index=["subject", "block", "m1", "m2", "uncertainty", "horizon"],
        columns="trial",
        values=feature
    ).rename(columns=lambda x: f"{prefix}{x}")

# Pivot reward, choice, RT
reward_wide = pivot_feature(df, "reward", "r")
choice_wide = pivot_feature(df, "choice", "c")
rt_wide = pivot_feature(df, "RT", "rt")

# Get static columns and merge all together
df_static = df.drop_duplicates(subset=["subject", "block", "m1", "m2", "uncertainty", "horizon"])
df_static = df_static.set_index(["subject", "block", "m1", "m2", "uncertainty", "horizon"])
df_wide = pd.concat([df_static, reward_wide, choice_wide, rt_wide], axis=1).reset_index()

df_wide.columns

Index(['subject', 'block', 'm1', 'm2', 'uncertainty', 'horizon', 'age',
       'gender', 'file_name', 'trial', 'reward', 'RT', 'choice', 'forced',
       'mask', 'r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'rt0',
       'rt1', 'rt2', 'rt3', 'rt4', 'rt5', 'rt6', 'rt7', 'rt8', 'rt9'],
      dtype='object')

In [37]:
df_wide

,subject,block,m1,m2,uncertainty,horizon,age,gender,file_name,trial,...,rt0,rt1,rt2,rt3,rt4,rt5,rt6,rt7,rt8,rt9
0,0,0,40,36,1,6,18,-99,LIFESPAN_students.csv,0,...,184.143661,4.635614,1.122532,2.833540,2.738855,5.263696,5.763985,1.940346,0.937745,2.472841
1,0,1,60,48,0,6,18,-99,LIFESPAN_students.csv,0,...,1.184903,1.013671,1.305989,8.874533,2.708196,2.426142,1.115581,1.370487,1.103393,1.046302
2,0,2,40,36,0,6,18,-99,LIFESPAN_students.csv,0,...,0.832413,0.468084,0.566725,0.445963,1.810017,1.325654,0.892086,0.780645,1.881181,0.576540
3,0,3,40,48,0,1,18,-99,LIFESPAN_students.csv,0,...,0.830383,0.553779,0.817720,1.686227,0.910277,NaN,NaN,NaN,NaN,NaN
4,0,4,40,10,0,1,18,-99,LIFESPAN_students.csv,0,...,0.663673,1.049479,0.692706,3.581879,4.616839,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170980,1032.0_TMS,155,39,35,-1,6,21,1,exp_TMS.csv,0,...,-99.000000,-99.000000,-99.000000,-99.000000,0.250419,0.250419,0.417128,0.283480,0.450473,0.316901
170981,1032.0_TMS,156,59,51,0,1,21,1,exp_TMS.csv,0,...,-99.000000,-99.000000,-99.000000,-99.000000,0.383629,NaN,NaN,NaN,NaN,NaN
170982,1032.0_TMS,157,63,63,0,1,21,1,exp_TMS.csv,0,...,-99.000000,-99.000000,-99.000000,-99.000000,0.217306,NaN,NaN,NaN,NaN,NaN
170983,1032.0_TMS,158,41,31,-1,1,21,1,exp_TMS.csv,0,...,-99.000000,-99.000000,-99.000000,-99.000000,0.150339,NaN,NaN,NaN,NaN,NaN


In [38]:
df = df_wide.rename(columns={"horizon": "gameLength", "uncertainty": "uc"})

df.columns

Index(['subject', 'block', 'm1', 'm2', 'uc', 'gameLength', 'age', 'gender',
       'file_name', 'trial', 'reward', 'RT', 'choice', 'forced', 'mask', 'r0',
       'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'c0', 'c1', 'c2',
       'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'rt0', 'rt1', 'rt2', 'rt3',
       'rt4', 'rt5', 'rt6', 'rt7', 'rt8', 'rt9'],
      dtype='object')

In [39]:
# df.to_csv("data/my_horizon_data_all_filenames.csv", index=False)

In [40]:
num_subjects = df["subject"].nunique()
print("Number of unique subjects:", num_subjects)

Number of unique subjects: 1384


In [41]:
# Collapse to one row per participant per file
subject_level = df.groupby(["file_name", "subject"]).agg({
    "age": "first",
    "gender": "first"     # 1 = female, 0 = male
}).reset_index()

# Summaries per file
summary = subject_level.groupby("file_name").agg(
    total_participants=("subject", "nunique"),
    age_min=("age", "min"),
    age_max=("age", "max"),
    age_mean=("age", "mean"),
    age_sd=("age", "std"),

    # Gender coding: 1 = female, 0 = male
    num_female=("gender", lambda x: (x == 1).sum()),
    num_male=("gender", lambda x: (x == 0).sum())
).reset_index()

print(summary)


                      file_name  total_participants  age_min  age_max  \
0  BATTERY_HorizonTaskFinal.csv                 234       18       74   
1            LIFESPAN_Harms.csv                 122      -99       38   
2       LIFESPAN_SZcontrols.csv                  38      -99       52   
3            LIFESPAN_Smith.csv                 418       18       40   
4      LIFESPAN_agingAZ_all.csv                 113      -99       74   
5      LIFESPAN_sommerville.csv                  88       12       17   
6         LIFESPAN_students.csv                 339      -99       50   
7                   exp_TMS.csv                  32      -99       32   

    age_mean     age_sd  num_female  num_male  
0  46.679487  23.938311          76       130  
1   6.450820  32.050606          40        72  
2  28.973684  32.209187          11        25  
3  23.715311   5.600000         303       115  
4  50.327434  30.417172          53        58  
5  14.897727   1.742239          46        42  
6  11.

In [ ]:
# Drop invalid ages (-99) BEFORE grouping
df_clean = df[df["age"] != -99].copy()
# df_clean = df

# Collapse to unique subjects
subject_level = df_clean.groupby(["file_name", "subject"]).agg({
    "age": "first",
    "gender": "first"
}).reset_index()

summary = subject_level.groupby("file_name").agg(
    total_participants=("subject", "nunique"),
    age_min=("age", "min"),
    age_max=("age", "max"),
    age_mean=("age", "mean"),
    age_sd=("age", "std"),
    num_female=("gender", lambda x: (x == 1).sum()),
    num_male=("gender", lambda x: (x == 0).sum())
).reset_index()

print(summary)

                      file_name  total_participants  age_min  age_max  \
0  BATTERY_HorizonTaskFinal.csv                 234       18       74   
1            LIFESPAN_Harms.csv                 122      -99       38   
2       LIFESPAN_SZcontrols.csv                  38      -99       52   
3            LIFESPAN_Smith.csv                 418       18       40   
4      LIFESPAN_agingAZ_all.csv                 113      -99       74   
5      LIFESPAN_sommerville.csv                  88       12       17   
6         LIFESPAN_students.csv                 339      -99       50   
7                   exp_TMS.csv                  32      -99       32   

    age_mean     age_sd  num_female  num_male  
0  46.679487  23.938311          76       130  
1   6.450820  32.050606          40        72  
2  28.973684  32.209187          11        25  
3  23.715311   5.600000         303       115  
4  50.327434  30.417172          53        58  
5  14.897727   1.742239          46        42  
6  11.

In [43]:
# Create the formatted age string
summary["age_info"] = summary.apply(
    lambda row: f"({row['age_min']}~{row['age_max']}, mean = {row['age_mean']:.2f}, SD = {row['age_sd']:.2f})",
    axis=1
)

# Select only the requested columns and rename
final_summary = summary[[
    "file_name",
    "total_participants",
    "num_female",
    "age_info"
]].rename(columns={
    "total_participants": "total_number",
    "num_female": "female_number"
})

print(final_summary)


                      file_name  total_number  female_number  \
0  BATTERY_HorizonTaskFinal.csv           234             76   
1            LIFESPAN_Harms.csv           122             40   
2       LIFESPAN_SZcontrols.csv            38             11   
3            LIFESPAN_Smith.csv           418            303   
4      LIFESPAN_agingAZ_all.csv           113             53   
5      LIFESPAN_sommerville.csv            88             46   
6         LIFESPAN_students.csv           339            100   
7                   exp_TMS.csv            32             18   

                             age_info  
0   (18~74, mean = 46.68, SD = 23.94)  
1   (-99~38, mean = 6.45, SD = 32.05)  
2  (-99~52, mean = 28.97, SD = 32.21)  
3    (18~40, mean = 23.72, SD = 5.60)  
4  (-99~74, mean = 50.33, SD = 30.42)  
5    (12~17, mean = 14.90, SD = 1.74)  
6  (-99~50, mean = 11.86, SD = 29.34)  
7  (-99~32, mean = 18.38, SD = 21.72)  


In [ ]:
# ---- Create TOTAL row ----
total_n = final_summary["total_number"].sum()
total_female = final_summary["female_number"].sum()
total_male = total_n - total_female

# Age summary across all rows
age_min = summary["age_min"].min()
age_max = summary["age_max"].max()
age_mean = summary["age_mean"].mean()   # NOTE: mean of means
age_sd = summary["age_sd"].mean()       # NOTE: mean of SDs

total_age_info = f"({age_min}~{age_max}, mean = {age_mean:.2f}, SD = {age_sd:.2f})"

# Build total row
total_row = pd.DataFrame({
    "file_name": ["TOTAL"],
    "total_number": [total_n],
    "female_number": [total_female],
    "age_info": [total_age_info]
})

# ---- Append total row to summary ----
final_summary_with_total = pd.concat([final_summary, total_row], ignore_index=True)

print(final_summary_with_total)


                      file_name  total_number  female_number  \
0  BATTERY_HorizonTaskFinal.csv           234             76   
1            LIFESPAN_Harms.csv           122             40   
2       LIFESPAN_SZcontrols.csv            38             11   
3            LIFESPAN_Smith.csv           418            303   
4      LIFESPAN_agingAZ_all.csv           113             53   
5      LIFESPAN_sommerville.csv            88             46   
6         LIFESPAN_students.csv           339            100   
7                   exp_TMS.csv            32             18   
8                         TOTAL          1384            647   

                             age_info  
0   (18~74, mean = 46.68, SD = 23.94)  
1   (-99~38, mean = 6.45, SD = 32.05)  
2  (-99~52, mean = 28.97, SD = 32.21)  
3    (18~40, mean = 23.72, SD = 5.60)  
4  (-99~74, mean = 50.33, SD = 30.42)  
5    (12~17, mean = 14.90, SD = 1.74)  
6  (-99~50, mean = 11.86, SD = 29.34)  
7  (-99~32, mean = 18.38, SD = 21.72)  